In [2]:
import pandas as pd
import numpy as np
import os

# Vérification de la présence des fichiers
for f in os.listdir():
    if "2023" in f and f.endswith(".csv"):
        print(f)

        

accidents_2022_2023_clean.csv
accidents_2022_2023_meteo.csv
accidents_2022_2023_quantile_clean.csv
accidents_2023_clean.csv
accidents_2023_final.csv
caract-2023.csv
lieux-2023.csv
usagers-2023.csv
vehicules-2023.csv


In [3]:
# Chargement des 4 fichiers BAAC 2022
carac_2022 = pd.read_csv("caract-2023.csv", sep=';',low_memory=False)
lieux_2022 = pd.read_csv("lieux-2023.csv", sep=';', low_memory=False)
veh_2022   = pd.read_csv("vehicules-2023.csv", sep=';', low_memory=False)
usag_2022  = pd.read_csv("usagers-2023.csv", sep=';', low_memory=False)

# Aperçu rapide des tailles et colonnes
for name, df in [("carac", carac_2022), ("lieux", lieux_2022), ("veh", veh_2022), ("usag", usag_2022)]:
    print(f"{name:8s} → {df.shape[0]:>7,} lignes × {df.shape[1]:>2} colonnes")
    print(f"Colonnes principales : {list(df.columns[:8])}")
    print("-" * 60)


carac    →  54,822 lignes × 15 colonnes
Colonnes principales : ['Num_Acc', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com']
------------------------------------------------------------
lieux    →  70,860 lignes × 18 colonnes
Colonnes principales : ['Num_Acc', 'catr', 'voie', 'v1', 'v2', 'circ', 'nbv', 'vosp']
------------------------------------------------------------
veh      →  93,585 lignes × 11 colonnes
Colonnes principales : ['Num_Acc', 'id_vehicule', 'num_veh', 'senc', 'catv', 'obs', 'obsm', 'choc']
------------------------------------------------------------
usag     → 125,789 lignes × 16 colonnes
Colonnes principales : ['Num_Acc', 'id_usager', 'id_vehicule', 'num_veh', 'place', 'catu', 'grav', 'sexe']
------------------------------------------------------------


In [4]:
#selection bonne caracteristique


import pandas as pd
import numpy as np

# 1️⃣ Renommage de la clé
carac_2022.rename(columns={'Accident_Id': 'Num_Acc'}, inplace=True)

# 2️⃣ Sélection des colonnes comme pour 2023
caract_prime_22 = carac_2022[[
    'Num_Acc', 'jour', 'mois', 'an', 'hrmn',
    'lat', 'long', 'com', 'dep',
    'lum', 'atm', 'agg', 'col'  # pas de 'adr' si absente
]].copy()

# Vérifie si 'adr' existe (en 2022 elle est souvent manquante)
if 'adr' in carac_2022.columns:
    caract_prime_22['adr'] = carac_2022['adr']

lieux_prime_22 = lieux_2022[[
    'Num_Acc', 'catr', 'circ', 'surf', 'infra', 'situ', 'plan', 'prof', 'vma'
]].copy()

vehicules_prime_22 = veh_2022[[
    'Num_Acc', 'catv', 'choc', 'manv'
]].copy()

usagers_prime_22 = usag_2022[[
    'Num_Acc', 'grav', 'sexe', 'an_nais', 'catu'
]].copy()

# 3️⃣ Vérification rapide des shapes
for name, df in [
    ("caract_prime_22", caract_prime_22),
    ("lieux_prime_22", lieux_prime_22),
    ("vehicules_prime_22", vehicules_prime_22),
    ("usagers_prime_22", usagers_prime_22)
]:
    print(f"{name:<18} → {df.shape[0]:>7,} lignes × {df.shape[1]} colonnes")

# 4️⃣ Vérifie que Num_Acc est unique dans la base principale
print("\nDoublons dans caract_prime_22 :", caract_prime_22['Num_Acc'].duplicated().sum())
print("Valeurs manquantes dans Num_Acc :", caract_prime_22['Num_Acc'].isna().sum())

# Aperçu
caract_prime_22.head(3)


caract_prime_22    →  54,822 lignes × 14 colonnes
lieux_prime_22     →  70,860 lignes × 9 colonnes
vehicules_prime_22 →  93,585 lignes × 4 colonnes
usagers_prime_22   → 125,789 lignes × 5 colonnes

Doublons dans caract_prime_22 : 0
Valeurs manquantes dans Num_Acc : 0


Num_Acc  jour  mois    an   hrmn          lat        long    com dep  \
0  202300000001     7     5  2023  06:00  48,86638600  2,32347100  75101  75   
1  202300000002     7     5  2023  05:30  48,84547782  2,42868146  94080  94   
2  202300000003     7     5  2023  20:50  48,76240000  2,40655000  94022  94   

   lum  atm  agg  col                        adr  
0    1    2    2    7              RUE DE RIVOLI  
1    5    3    2    6            Avenue de Paris  
2    1    2    2    1  Avenue du Général Leclerc

In [5]:
import numpy as np
import pandas as pd

# ============================================================
# 1️⃣ LIEUX : agrégation (1 ligne = 1 accident)
# Hypothèse : le lieu est identique pour toutes les lignes
# ============================================================

lieux_prime_22["vma"] = lieux_prime_22["vma"].replace(-1, np.nan)

lieux_agg_22 = lieux_prime_22.groupby("Num_Acc", as_index=False).agg({
    "catr": "first",
    "circ": "first",
    "surf": "first",
    "infra": "first",
    "situ": "first",
    "plan": "first",
    "prof": "first",
    "vma": lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
})

print("✅ lieux_agg_22 :", lieux_agg_22.shape)


# ============================================================
# 2️⃣ VÉHICULES : regroupement EXHAUSTIF par grandes familles
# ============================================================

# Groupes couvrant 100 % des valeurs BAAC de catv
VEH_GROUPS = {
    "has_2RM":   [30, 31, 32, 33, 34, 41, 42, 43],
    "has_VL":    [3, 7, 10, 35, 36, 80],
    "has_PL":    [13, 14, 15, 16, 17, 21],
    "has_TC":    [37, 38, 39, 40],
    "has_MD":    [1, 2, 50, 60],
    "has_OTHER": [0, 20, 99]
}

vehicules_prime_22["catv"] = vehicules_prime_22["catv"].fillna(-1)

# Nombre total de véhicules
vehicules_agg_22 = (
    vehicules_prime_22
    .groupby("Num_Acc")
    .agg(nb_vehicules=("Num_Acc", "count"))
    .reset_index()
)

# Variables de présence par groupe
for col, codes in VEH_GROUPS.items():
    tmp = (
        vehicules_prime_22
        .assign(flag=lambda x: x["catv"].isin(codes).astype(int))
        .groupby("Num_Acc")["flag"]
        .max()
        .reset_index()
        .rename(columns={"flag": col})
    )
    vehicules_agg_22 = vehicules_agg_22.merge(tmp, on="Num_Acc", how="left")

vehicules_agg_22.fillna(0, inplace=True)

print("✅ vehicules_agg_22 :", vehicules_agg_22.shape)


# ============================================================
# 3️⃣ CHOC : information simple mais pertinente
# ============================================================

choc_agg_22 = (
    vehicules_prime_22
    .assign(choc_multiple=lambda x: (x["choc"] == 9).astype(int))
    .groupby("Num_Acc")["choc_multiple"]
    .max()
    .reset_index()
)

vehicules_agg_22 = vehicules_agg_22.merge(choc_agg_22, on="Num_Acc", how="left")
vehicules_agg_22["choc_multiple"].fillna(0, inplace=True)


# ============================================================
# 4️⃣ USAGERS : nettoyage âge + agrégation
# ============================================================

# Calcul âge
usagers_prime_22["age"] = 2022 - usagers_prime_22["an_nais"]

# Suppression des âges aberrants
usagers_prime_22.loc[
    (usagers_prime_22["age"] < 0) | (usagers_prime_22["age"] > 100),
    "age"
] = np.nan

usagers_agg_22 = usagers_prime_22.groupby("Num_Acc", as_index=False).agg(
    nb_usagers=("Num_Acc", "count"),
    grav_max=("grav", "max"),     # variable cible
    age_moyen=("age", "mean")
)

print("✅ usagers_agg_22 :", usagers_agg_22.shape)


# ============================================================
# 5️⃣ Vérification de cohérence avec la table principale
# ============================================================

for name, df in [
    ("lieux_agg_22", lieux_agg_22),
    ("vehicules_agg_22", vehicules_agg_22),
    ("usagers_agg_22", usagers_agg_22)
]:
    inter = len(set(df["Num_Acc"]) & set(caract_prime_22["Num_Acc"]))
    print(f"{name:<18} → correspondances : {inter:,}/{len(df):,}")

display(lieux_agg_22.head(3))
display(vehicules_agg_22.head(3))
display(usagers_agg_22.head(3))


✅ lieux_agg_22 : (54822, 9)
✅ vehicules_agg_22 : (54822, 8)
✅ usagers_agg_22 : (54822, 4)
lieux_agg_22       → correspondances : 54,822/54,822
vehicules_agg_22   → correspondances : 54,822/54,822
usagers_agg_22     → correspondances : 54,822/54,822


C:\Users\Windows\AppData\Local\Temp\ipykernel_3656\3031176143.py:79: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  vehicules_agg_22["choc_multiple"].fillna(0, inplace=True)


Num_Acc  catr  circ  surf  infra  situ  plan  prof   vma
0  202300000001     4     1     2      0     1     1     1  30.0
1  202300000002     3     2     2      0     1     1     1  50.0
2  202300000003     3     2     2      5     1     1     1  50.0

Num_Acc  nb_vehicules  has_2RM  has_VL  has_PL  has_TC  has_MD  \
0  202300000001             1        1       0       0       0       0   
1  202300000002             1        0       1       0       0       0   
2  202300000003             2        0       1       0       0       1   

   has_OTHER  choc_multiple  
0          0              0  
1          0              0  
2          0              0

Num_Acc  nb_usagers  grav_max  age_moyen
0  202300000001           1         4       44.0
1  202300000002           2         3       25.0
2  202300000003           2         3       36.5

In [6]:
# ============================================================
# FUSION FINALE 2022 (APRÈS AGRÉGATIONS)
# ============================================================

df_2022 = (
    caract_prime_22
    .merge(lieux_agg_22, on="Num_Acc", how="left")
    .merge(vehicules_agg_22, on="Num_Acc", how="left")
    .merge(usagers_agg_22, on="Num_Acc", how="left")
)

print(f"✅ Fusion réussie : {df_2022.shape[0]} lignes, {df_2022.shape[1]} colonnes")
print(f"Accidents uniques : {df_2022['Num_Acc'].nunique()}")

display(df_2022.head(3))


# ============================================================
# SAUVEGARDE DU JEU 2022 AGRÉGÉ
# ============================================================

output_path_2022 = "./accidents_2023_finalV3.csv"
df_2022.to_csv(output_path_2022, index=False, sep=";", encoding="utf-8")

print(f"✅ Fichier 2022 enregistré : {output_path_2022}")
print(f"Lignes : {len(df_2022)} | Colonnes : {len(df_2022.columns)}")


✅ Fusion réussie : 54822 lignes, 33 colonnes
Accidents uniques : 54822


Num_Acc  jour  mois    an   hrmn          lat        long    com dep  \
0  202300000001     7     5  2023  06:00  48,86638600  2,32347100  75101  75   
1  202300000002     7     5  2023  05:30  48,84547782  2,42868146  94080  94   
2  202300000003     7     5  2023  20:50  48,76240000  2,40655000  94022  94   

   lum  ...  has_2RM  has_VL  has_PL has_TC  has_MD  has_OTHER  choc_multiple  \
0    1  ...        1       0       0      0       0          0              0   
1    5  ...        0       1       0      0       0          0              0   
2    1  ...        0       1       0      0       1          0              0   

   nb_usagers  grav_max  age_moyen  
0           1         4       44.0  
1           2         3       25.0  
2           2         3       36.5  

[3 rows x 33 columns]

✅ Fichier 2022 enregistré : ./accidents_2023_finalV3.csv
Lignes : 54822 | Colonnes : 33
